In [64]:
import torch
from torchvision import models as models
import numpy as np
import gc
import time
import glob
from torchvision import transforms, datasets
import random
from torch.utils.data import DataLoader, Subset

In [65]:
def clear_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    elif torch.backends.mps.is_available():
        torch.mps.empty_cache()

In [66]:
def create_results_file(filename=f'pruning_results_{time.time()}.csv'):
    with open(filename, 'w') as f:
        f.write("Pruning Amount, Final Accuracy, Time, Memory\n")
    return filename

In [67]:
def append_result(filename, pruning_amount, accuracy, compute_time, model_size):
    with open(filename, 'a') as f:
        f.write(f"{pruning_amount}%, {accuracy}%,{compute_time}, {model_size}\n")

In [68]:
model_paths = glob.glob("AlexNetPruned/*")
model_paths.sort()

In [69]:
def get_images(folder_path='imagenet-mini/train', num_samples=200):
    transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(), 
        transforms.RandomRotation(15),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                            std=[0.229, 0.224, 0.225]),
    ])
    data_dataset = datasets.ImageFolder(folder_path, transform=transform)
    indices = random.sample(range(len(data_dataset)), min(num_samples, len(data_dataset)))
    data_dataset = Subset(data_dataset, indices)
    
    return DataLoader(data_dataset, batch_size=32, shuffle=True, num_workers=1, pin_memory=True)


In [70]:
def test(model, test_loader, device, final_test=True):
    model.eval()
    model.to(device)
    correct_top1 = 0
    total = 0
    compute_time = 0
    accuracies = []
    computation_times = []
    
    tries = 3 if final_test else 1
    for _ in range(tries):        
        with torch.inference_mode():
            for images, labels in test_loader:
                try:
                    images = images.to(device, non_blocking=True)
                    labels = labels.to(device, non_blocking=True)
                    
                    t1 = time.time()
                    outputs = model(images)
                    t2 = time.time()
                    compute_time += t2 - t1
                    
                    # Top-1 accuracy
                    _, predicted = torch.max(outputs, 1)
                    total += labels.size(0)
                    correct_top1 += (predicted == labels).sum().item()
                    
                finally:
                    # Free up memory
                    del images, labels
                    if 'outputs' in locals(): 
                        del outputs
                    if 'predicted' in locals(): 
                        del predicted
                    clear_memory()
        
        accuracies.append(100.0 * correct_top1 / total if total > 0 else 0)
        computation_times.append(compute_time)
        clear_memory()
    del test_loader
    clear_memory()
    
    return [np.mean(accuracies), min(computation_times)]

In [71]:
def get_model_size(model):
    total_size = sum(
        param.nelement() * param.element_size() for param in model.parameters()
    )
    # Convert to MB
    return total_size / (1024 ** 2)

In [72]:
print(model_paths)

['AlexNetPruned/pruned_AlexNet_00.00.pth', 'AlexNetPruned/pruned_AlexNet_00.50.pth', 'AlexNetPruned/pruned_AlexNet_01.00.pth', 'AlexNetPruned/pruned_AlexNet_01.50.pth', 'AlexNetPruned/pruned_AlexNet_02.00.pth', 'AlexNetPruned/pruned_AlexNet_02.50.pth', 'AlexNetPruned/pruned_AlexNet_03.00.pth', 'AlexNetPruned/pruned_AlexNet_03.50.pth', 'AlexNetPruned/pruned_AlexNet_04.00.pth', 'AlexNetPruned/pruned_AlexNet_04.50.pth', 'AlexNetPruned/pruned_AlexNet_05.00.pth', 'AlexNetPruned/pruned_AlexNet_05.50.pth', 'AlexNetPruned/pruned_AlexNet_06.00.pth', 'AlexNetPruned/pruned_AlexNet_06.50.pth', 'AlexNetPruned/pruned_AlexNet_07.00.pth', 'AlexNetPruned/pruned_AlexNet_07.50.pth', 'AlexNetPruned/pruned_AlexNet_08.00.pth', 'AlexNetPruned/pruned_AlexNet_08.50.pth', 'AlexNetPruned/pruned_AlexNet_09.00.pth', 'AlexNetPruned/pruned_AlexNet_09.50.pth', 'AlexNetPruned/pruned_AlexNet_10.00.pth', 'AlexNetPruned/pruned_AlexNet_10.50.pth', 'AlexNetPruned/pruned_AlexNet_11.00.pth', 'AlexNetPruned/pruned_AlexNet_11.

In [63]:
filename = create_results_file(input("Enter filename for results: ")+".csv")

# device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
device = 'cpu'
print(f"Using device: {device}")


# Create a test dataset
test_loader = get_images()

for model_path in model_paths:
    pruning_amount = model_path.split('_')[-1].replace('.pth', '')
    print(f"Testing model with pruning amount {pruning_amount}%")
    
    # Load the pruned model
    model = torch.load(model_path, weights_only=False, map_location=device)
    
    # Get the size of the model
    model_size = get_model_size(model)
    
    # Test the model
    accuracy, compute_time = test(model, test_loader, device, final_test=True)
    
    pruning_amount = str(model_path.split('_')[-1].split('.')[0]) + '.' + str(model_path.split('_')[-1].split('.')[1])
    
    print(f"Pruning Amount: {pruning_amount}%")
    # Append results to file
    append_result(filename, pruning_amount, accuracy, compute_time, model_size)
    print(f"Model size: {model_size:.2f} MB")
    print(f"Final accuracy: {accuracy:.2f}%")
    print(f"Compute time: {compute_time:.5f} seconds")
    clear_memory()
    del model
    clear_memory()
    print("Memory cleared.")
    print("========================================")

Using device: cpu
Testing model with pruning amount 0.00%
Pruning Amount: 0.00%
Model size: 233.08 MB
Final accuracy: 54.31%
Compute time: 1.06045 seconds
Memory cleared.
Testing model with pruning amount 0.50%
Pruning Amount: 0.50%
Model size: 232.44 MB
Final accuracy: 56.67%
Compute time: 0.95516 seconds
Memory cleared.
Testing model with pruning amount 1.00%
Pruning Amount: 1.00%
Model size: 231.75 MB
Final accuracy: 50.42%
Compute time: 0.94975 seconds
Memory cleared.
Testing model with pruning amount 1.50%
Pruning Amount: 1.50%
Model size: 229.98 MB
Final accuracy: 58.69%
Compute time: 0.94410 seconds
Memory cleared.
Testing model with pruning amount 10.00%
Pruning Amount: 10.00%
Model size: 219.49 MB
Final accuracy: 49.72%
Compute time: 0.90458 seconds
Memory cleared.
Testing model with pruning amount 10.50%
Pruning Amount: 10.50%
Model size: 217.78 MB
Final accuracy: 53.56%
Compute time: 0.89823 seconds
Memory cleared.
Testing model with pruning amount 11.00%
Pruning Amount: 11.